In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Importing and preprocessing

In [3]:
df_total = pd.read_csv("../leo_kaggle/data.csv")
df_total;

In [4]:
df_total["months"] = df_total["game_date"].apply(lambda x:x.split('-')[1])
df_total["years"] = df_total["game_date"].apply(lambda x:x.split('-')[0])
df_trans = df_total[df_total["shot_made_flag"] * 1 == df_total["shot_made_flag"]]
# We take only a tenth of the set to play with to have a better speed

df = df_trans.iloc[:len(df_trans)]

df_test = df_total[df_total["shot_made_flag"] * 1 != df_total["shot_made_flag"]]
print(len(df))

25697


In [5]:
from sklearn import preprocessing

In [6]:
def cartesian_to_polar(df, name_x, name_y):
    df["loc_r"]=df.apply(lambda row:np.sqrt(row[name_x]**2 + row[name_y]**2), axis =1)
    df["loc_teta"]= df.apply(lambda row:np.arccos(row[name_x] / row["loc_r"]) if row["loc_r"] else 0, axis=1)

def drop_and_binarize(df, to_binarize, to_drop):
    new_df = df.copy()
    to_binarize =[feature for feature in to_binarize if feature not in to_drop]
    print(len(new_df.columns))
    new_df.drop(to_drop, axis=1, inplace=True)
    print(len(new_df.columns))
    lb = preprocessing.LabelBinarizer()
    for feature in to_binarize:
        lb.fit(df[feature])
        new_df[feature]=df[feature].apply(lambda x:lb.transform([x]).flatten())
    return new_df


In [7]:
def aux_df_to_array(ar):
    l = list()
    for elem in ar:
            if type(elem) is numpy.ndarray:
                l.extend(elem)
            else:
                l.append(elem)
    return np.array(l)

def df_to_array(df):
    return np.apply_along_axis(aux_df_to_array, 1, df.values)

def indice_to_name(df):
    return np.array([[key for i in range(len(np.unique(df[key])))] for key in df.columns]).flatten()

In [8]:
def rescale(vect_train, min_max):
    res = preprocessing.MinMaxScaler(min_max)
    return res.fit_transform(vect_train)

def standardize(vect_train):
    sta = preprocessing.StandardScaler()
    return sta.fit_transform(vect_train)

In [9]:
def n_shots_before(shot):
    game_id = shot["game_id"]
    shots = df[df["game_id"] == game_id]
    mask = (shots["minutes_remaining"] == shot["minutes_remaining"]) & (shots["seconds_remaining"] > shot["seconds_remaining"]) | (shots["minutes_remaining"] > shot["minutes_remaining"])
    if shot["shot_id"]%1000 ==0:
        print(shot["shot_id"])
    return shots[mask]["shot_made_flag"].count(), shots[mask]["shot_made_flag"].sum()

def n_successive_shots(shot):
    game_id = shot["game_id"]
    shots = df[df["game_id"] == game_id]
    shots = shots.sort_values(by=["minutes_remaining", "seconds_remaining"], ascending=False).reset_index()
    #On trouve l'index de shot
    index = 0
    for i, a in enumerate(shots["shot_id"]==shot["shot_id"]):
        if a:
            index = i
    result = 0
    for i in range(index - 1, -1, -1):
        if not shots.iloc[i]["shot_made_flag"] :
            result = index - i -1
            break
        elif not i :
            result = index
    return result

def add_on_fire_features(df):
    col = df.apply(n_shots_before, axis =1)
    col2 = df.apply(n_successive_shots, axis =1)
    #df["n_successive_shots"] = col2.reset_index(drop=True)
    #df["n_shots_tried_before"]=col.apply(lambda x:x[0]).reset_index(drop=True)
    #df["n_shots_made_before"]=col.apply(lambda x:x[1]).reset_index(drop=True)
    #df["previous_win_frequency"]=col.apply(lambda x: (x[1] / x[0]) if x[0]>0 else 0 ).reset_index(drop=True)
    df["n_successive_shots"] = col2
    df["n_shots_tried_before"]=col.apply(lambda x:x[0])
    df["n_shots_made_before"]=col.apply(lambda x:x[1])
    df["previous_win_frequency"]=col.apply(lambda x: (x[1] / x[0]) if x[0]>0 else 0 )

In [10]:
cartesian_to_polar(df, "loc_x", "loc_y")

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
add_on_fire_features(df)

In [12]:
features_to_binarize = ["action_type", "combined_shot_type", "shot_type", "shot_zone_area", "shot_zone_basic", "shot_zone_range", "team_name", "matchup", "opponent", "months", "years", "team_id", "game_event_id", "season"]
#features_to_drop = ["playoffs", "team_id", "team_name", "opponent", "lat", "lon", "season", "shot_zone_basic", "shot_zone_range", "shot_distance"]
features_to_drop = [ "team_name", "shot_id", "game_date", "game_id"]

In [13]:
new_df = drop_and_binarize(df, features_to_binarize, features_to_drop)
#new_df.reset_index()

29
25


In [14]:
new_df.drop(["game_event_id"], axis=1, inplace=True)

In [15]:
new_df.drop(["matchup"], axis=1, inplace=True)
new_df.drop(["team_id"], axis=1, inplace=True)

In [16]:
vect_df_target = new_df["shot_made_flag"].values
new_df = drop_and_binarize(new_df, [], ["shot_made_flag"])

22
21


In [17]:
def len_extended(thing):
    try:
        return len(thing)
    except TypeError:
        return 1
    
def indice_to_name(df):
    l = list()
    for key in df.columns:
        l.extend([key for i in range(len_extended(df[key][1]))])
    return np.array(l)
traducteur = indice_to_name(new_df)

In [18]:
vect_df_features = df_to_array(new_df)   

In [19]:
from sklearn.model_selection import train_test_split
vect_train_features, vect_test_features, vect_train_target, vect_test_target = train_test_split(vect_df_features, vect_df_target, test_size=0.2, shuffle=True)
print(vect_train_features.shape)
vect_train_features = standardize(vect_train_features)
vect_test_features = standardize(vect_test_features)


(20557, 174)


In [22]:
vect_train_features.shape, vect_train_target.shape

((20557, 174), (20557,))

vect_train_features = vect_df_features[: int(len(vect_df_features) * 0.8)]
vect_test_features = vect_df_features[int(len(vect_df_features) * 0.8) :]
vect_train_target = vect_df_target[: int(len(vect_df_features) * 0.8)]
vect_test_target = vect_df_target[int(len(vect_df_features)*0.8):]
vect_train_features = standardize(vect_train_features)
vect_test_features = standardize(vect_test_features)
#vect_train_features = rescale(vect_train_features, (0,1))
#vect_test_features = rescale(vect_test_features, (0, 1))


In [20]:
#vect_train_features, vect_test_features = vect_test_features, vect_train_features

In [21]:
#vect_train_target, vect_test_target = vect_test_target, vect_train_target

# Now let's feed it to our nn

In [22]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import functools

In [23]:
def init_weights(m, i):
    if type(m) == nn.Linear:
        m.weight.data.fill_(i)

def init_weight_normal(neuron, mean, std):
    if type(neuron) == nn.Linear:
        neuron.weight.data.normal_(mean, std)

def init_weight_relu(neuron):
    if type(neuron) == nn.Linear:
        n = len(neuron.weight.data)
        neuron.weight.data.normal_(0, np.sqrt(2. / n))
        
def init_normalized_bengio(neuron):
    if type(neuron) == nn.Linear:
        n_output, n_input = neuron.weight.data.numpy().shape
        neuron.weight.data.uniform_(0, np.sqrt(6 / (n_input + n_output)))
        
def init_uniform(neuron):
    if type(neuron) == nn.Linear:
        n_output, n_input = neuron.weight.data.numpy().shape
        neuron.weight.data.uniform_(0, np.sqrt(1 / (n_input)))
        
def normalize(vect):
    vect = (vect - np.mean(vect)) / np.std(vect)
    return vect

    
def weight_scaling_inference_rule(net, p_init, p_hidden):
    """Multiply the weight of each layer by its probability of not being droppped out during training to average bagging"""
    list(net.children())[0].weight.data *= (1-p_init)
    for neuron in list(net.children())[1:]:
        if neuron.type == nn.Linear:
            neuron.weight.data *= (1-p_hidden)
        

In [24]:

class Net4(nn.Module):
    
    def __init__(self, n_features, n_classes, dropout=(0, 0)):
        super(Net4, self).__init__()
        self.p_init = dropout[0]
        self.p_hidden = dropout[1]
        self.fc1 = nn.Linear(n_features, 20, True)
        self.fc2 = nn.Linear(20, 20, True)
        self.fc3 = nn.Linear(20, 20, True)
        self.fc4 = nn.Linear(20, 20, True)
        self.fc5 = nn.Linear(20, 10, True)
        self.fc6 = nn.Linear(10, 20, True)
        self.fc7 = nn.Linear(20, 20, True)
        self.fc11 = nn.Linear(20, 20, True)
        self.fc12 = nn.Linear(20, 20, True)
        self.fc13 = nn.Linear(20, n_classes, True)
        self.fc14 = nn.LogSoftmax()
        self.criterion = nn.NLLLoss()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(F.dropout(self.fc1(x), p=self.p_init, training=self.training))
        x = F.relu(F.dropout(self.fc2(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc3(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc4(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc5(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc6(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc7(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc11(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc12(x), p=self.p_hidden, training=self.training))
        x = self.fc13(x)
        x = self.fc14(x)
        return x


In [25]:

class Net5(nn.Module):
    
    def __init__(self, n_features, n_classes, dropout=(0, 0)):
        super(Net5, self).__init__()
        self.p_init = dropout[0]
        self.p_hidden = dropout[1]
        self.fc1 = nn.Linear(n_features, 20, True)
        self.fc2 = nn.Linear(20, 20, True)
        self.fc3 = nn.Linear(20, 20, True)
        self.fc4 = nn.Linear(20, 20, True)
        self.fc13 = nn.Linear(20, n_classes, True)
        self.fc14 = nn.LogSoftmax()
        self.criterion = nn.NLLLoss()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(F.dropout(self.fc1(x), p=self.p_init, training=self.training))
        x = F.relu(F.dropout(self.fc2(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc3(x), p=self.p_hidden, training=self.training))
        x = F.relu(F.dropout(self.fc4(x), p=self.p_hidden, training=self.training))
        x = self.fc13(x)
        x = self.fc14(x)
        return x



In [26]:
from sklearn.base import BaseEstimator, ClassifierMixin
import copy

class MultiInitialisationClassifier(BaseEstimator, ClassifierMixin):
    """ Sklearn-compatible classifier that implement a pytorch feedforward neural network 
    and tries several weight initialisation in its "fit" method"""
    
    
    def __init__(self, net_class=nn.Sequential, criterion=nn.NLLLoss(), batch_size=32, n_epochs=10, 
                 n_inits_per_optimizer=15, max_fail=4, dropout = (0, 0), init_function=init_weight_relu, validation_set=None,
                 optimizer_classes={"Adadelta":functools.partial(optim.Adadelta, weight_decay=0.01)}):
        
        self.trained_net = None
        self.dropout = dropout
        self.p_init = dropout[0]
        self.p_hidden = dropout[1]
        self.criterion = criterion
        self.net_class = net_class
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.n_inits_per_optimizer= n_inits_per_optimizer
        self.max_fail= max_fail
        self.optimizer_classes = optimizer_classes
        self.init_function = init_function
        self.validation_set = validation_set
    
    def multi_init(self):
        net_list = list()
        for i, key in enumerate(self.optimizer_classes.keys()):
            for k in range(self.n_inits_per_optimizer):
                net = self.net_class_init()       
                net.apply(self.init_function)
                #net.apply(init_uniform)
                net.train()
                net_list.append([net, self.optimizer_classes[key](net.parameters()), 0.0])
        return net_list
    
    def load_data(self, dataset):
        trainloader = DataLoader(dataset, batch_size = self.batch_size, shuffle=True)
        return trainloader
    
    def fit(self, X, y, n_epochs=None,
            validation_set=None, max_fail=5, thresold_fail=1.5):
        
        print(self.dropout)
        print(self.batch_size)
        print(self.optimizer_classes)
        self.p_init = self.dropout[0]
        self.p_hidden = self.dropout[1]
        self.net_class_init=functools.partial(self.net_class, n_features=X.shape[1], n_classes=len(np.unique(y)),dropout=self.dropout)
        if n_epochs == None:
            n_epochs = self.n_epochs
        if validation_set == None:
            validation_set = self.validation_set

        ## Load the data
        input, target = torch.from_numpy(X), torch.from_numpy(y)
        dataset = TensorDataset(input, target)
        trainloader = self.load_data(dataset)
        
        ## Create the different initialisation (i.e create different nets)
        net_list = self.multi_init()


        ## Train the nets
        print("BEGINNING TRAINING")
        loss_list = list()
        loss_list_validation = list()
        best_net_on_validation = [None, np.inf]
        for epoch in range(n_epochs):
            for net in net_list :
                net[2] = 0.0
                n_iter = 0
            for data in trainloader:
                input, target = data
                target = target.view(-1)
                input, target = Variable(input).float(), Variable(target).long()
                for i in range(len(net_list)):
                    net, optimizer, _ = net_list[i]
                    optimizer.zero_grad()
                    output = net(input)
                    loss = self.criterion(output, target)
                    loss.backward()
                    optimizer.step()
                    net_list[i][2] += loss.data[0]
                n_iter += 1
            index_min = np.nanargmin([net[2] for net in net_list])
            print(net_list[index_min][2] / n_iter)
            loss_list.append(net_list[index_min][2] / n_iter)
            ## Check the loss on the validation set and stop if it's increasing for at least max_fail epochs 
            #  or go back above thresold_fail * (minimum loss attained on the validation set)
            if validation_set:
                # copy the best net and make it in evaluation mode for scoring
                best_net = copy.deepcopy(net_list[index_min][0])
                weight_scaling_inference_rule(best_net, self.p_init, self.p_hidden)
                best_net.eval()
                score = self.score(validation_set[0], validation_set[1], best_net)
                print(score)
                loss_list_validation.append(score)
                if score < best_net_on_validation[1]: #WARNING : loss or score ???
                    best_net_on_validation = [best_net, score]
                if (score > thresold_fail * best_net_on_validation[1] or
                    max_fail < len(loss_list_validation) 
                    and (np.array([loss_list_validation[i] - loss_list_validation[i-1] for i in range(- 1, - max_fail - 1, - 1)]) > 0).all()
                    ) :
                    print("EARLY STOPPING")
                    self.trained_net = best_net_on_validation[0]
                    return loss_list, loss_list_validation
            ##
            
            if len(net_list) > 1:
                del net_list[np.argmax([net[2] for net in net_list])]
            print(epoch)
            
        self.trained_net = best_net_on_validation[0]
        l = loss_list, loss_list_validation #before
        return self #for sklearn
    
    
    def predict(self, X):
        if not self.trained_net:
            raise RuntimeError("You must train the classifier before predicting !")
        previous_state = self.trained_net.training
        self.trained_net.eval()
        input = Variable(torch.from_numpy(X)).float()
        output = self.trained_net(input)
        predicted = torch.max(output.data, 1)[1]
        self.trained_net.training = previous_state
        return predicted.numpy()
    
        
    def predict_proba(self, X, net=None):
        if not net and  not self.trained_net:
            raise RuntimeError("You must train the classifier before predicting !")
        if not net:
            net = self.trained_net
        previous_state = net.training
        net.eval()
        input = Variable(torch.from_numpy(X)).float()
        output = net(input)
        predicted = output.data
        net.training = previous_state
        return predicted.numpy()
    
    def score(self, X, y, net=None):
        if not net and  not self.trained_net:
            raise RuntimeError("You must train the classifier before scoring !")
        if not net:
            net = self.trained_net
        previous_state = net.training
        net.eval()
        input, target = Variable(torch.from_numpy(X)).float(), Variable(torch.from_numpy(y)).long()
        output = net(input)
        net.training = previous_state
        return self.criterion(output, target).data[0]

            

            
        

In [27]:
%%time
optimizer_classes = { #"SGD lr=0.1 momentum=0.9":functools.partial(optim.SGD, lr=1, momentum=0.9, weight_decay=0.01), 
                         #"Adam lr=0.1":functools.partial(optim.Adam, lr=0.1, weight_decay=0.01), 
                         "Adadelta":functools.partial(optim.Adadelta, weight_decay=0.01),
                         #"RMSprop lr=0.02":functools.partial(optim.RMSprop, lr=1.2, weight_decay=0.01)
                        }
clf = MultiInitialisationClassifier(Net5, nn.NLLLoss(), dropout=(0.2, 0.5), optimizer_classes=optimizer_classes, n_epochs = 30 , n_inits_per_optimizer = 40, batch_size=512,
       validation_set=[vect_test_features,vect_test_target], max_fail = 4)
l = clf.fit(vect_train_features, vect_train_target, max_fail = 5, thresold_fail = 1.5)

(0.2, 0.5)
512
{'Adadelta': <functools.partial object at 0x7f67f9022cb0>}
BEGINNING TRAINING
3.99268066011
0.74940431118
0
1.38419383328
0.689600110054
1
0.881436833521
0.68711566925
2
0.737447772084
0.687489151955
3


KeyboardInterrupt: 

In [28]:
output0, target0 = clf.predict_proba(vect_test_features), vect_test_target
output = Variable(torch.from_numpy(output0)).float()
target = Variable(torch.from_numpy(target0)).long()
criterion = nn.NLLLoss()
loss = criterion(output, target)
print(loss)

RuntimeError: You must train the classifier before predicting !

In [ ]:
output0, target0 = clf.predict_proba(vect_train_features), vect_train_target
output = Variable(torch.from_numpy(output0)).float()
target = Variable(torch.from_numpy(target0)).long()
criterion = nn.NLLLoss()
loss = criterion(output, target)
print(loss)

In [ ]:
predicted = clf.predict(vect_train_features).reshape(-1)
expected = vect_train_target
print( "{} correct points for {} points : accuracy = {}".format(sum(predicted==expected), len(predicted), float(sum(predicted==expected))/ len(predicted)))

In [ ]:
predicted = clf.predict(vect_test_features).reshape(-1)
expected = vect_test_target
print( "{} correct points for {} points : accuracy = {}".format(sum(predicted==expected), len(predicted), float(sum(predicted==expected))/ len(predicted)))

In [ ]:
train_loss_list = l[0]
test_loss_list = l[1]
print(min(train_loss_list))
print(min(test_loss_list))
epochs = [i for i in range(len(train_loss_list))]
f, ax = plt.subplots(2)
f.subplots_adjust(wspace=0.5, hspace=0.5)
ax[0].set_title("train loss")
ax[0].plot(epochs, train_loss_list)
ax[1].set_title("test_loss")
ax[1].plot(epochs, test_loss_list)


In [ ]:
a = np.argsort(np.sum(np.abs(clf.trained_net.fc1.weight.data.numpy()), axis=0))[::-1]
#c =  np.argsort(np.sum(np.abs(clf.trained_net.fc1.bias.data.numpy() + np.abs(clf.trained_net.fc1.weight.data.numpy()), axis=0))[::-1])
print(traducteur[a])
#print(traducteur[c])

# Pipeling and grid-searching

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
weight_decay_list = [ 0.01, 0.1]
dropout_list = [(0, 0), (0.2, 0.5), (0, 0.5), (0.2, 0.7), (0, 0.7)]
#n_inits_per_optimizer_list = [12, 50]
batch_size_list = [8, 128, 512, 2048]
optimizer_classes_list = [{"Adadelta":functools.partial(optim.Adadelta, weight_decay=i)} for i in weight_decay_list]
optimizer_classes_list.append({"Adam":functools.partial(optim.SGD, lr=3, weight_decay=0.01)})
optimizer_classes_list.append({"SGD":functools.partial(optim.SGD, lr=2.5, momentum=0.99, weight_decay=0.01)})

print(len(optimizer_classes_list))
print(len(dropout_list))
print( len(optimizer_classes_list) * len(batch_size_list)  * len(dropout_list))


optimizer_classes = { "SGD lr=0.1 momentum=0.9":functools.partial(optim.SGD, lr=0.2, momentum=0.9), 
                         "Adam lr=0.1":functools.partial(optim.Adam, lr=0.2), 
                         "Adadelta":functools.partial(optim.Adadelta),
                         "RMSprop lr=0.02":functools.partial(optim.RMSprop, lr=0.05)
                        }

clf = MultiInitialisationClassifier(Net5, nn.NLLLoss())
pipe = Pipeline([("preprocessing", StandardScaler), ("optimizer_classes", svm)])

In [ ]:
%%time
parameters = {'optimizer_classes':optimizer_classes_list, 'dropout':dropout_list, 'batch_size':batch_size_list}
clf = MultiInitialisationClassifier(net_class=Net5, criterion=nn.NLLLoss(), n_epochs = 40 , n_inits_per_optimizer = 30)
clf = GridSearchCV(clf, parameters, cv=2, n_jobs=-1)
clf.fit(vect_df_features, vect_df_target)

In [ ]:
parameters = {'optimizer_classes':optimizer_classes_list, 'dropout':dropout_list}
